In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Reshape
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
from keras.optimizers import Adam
import random

# Define the file paths for each period
file_paths = {
    'bear_market_1': '/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bear_market_1_classification.csv',
    'bear_market_2': '/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bear_market_2_classification.csv',
    'bull_market_1': '/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bull_market_1_classification.csv',
    'bull_market_2': '/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bull_market_2_classification.csv'
}

# Initialize the results dictionary
results = {}

# Seed value for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Normalize the data
columns_to_scale = ['Open', 'High', 'Low', 'Close', 'Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users']

scaler = MinMaxScaler(feature_range=(0, 1))

# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# Forecast Horizon
window_size = 3
close_index = 22

# Define a model-building function
def build_model(hp):
    model = Sequential()
    filters = hp.Int('filters', min_value=50, max_value=100, step=5)
    pool = hp.Choice('pool_size', values=[2, 3])
    model.add(Conv2D(
        filters=filters,
        kernel_size=3,
        activation='relu',
        input_shape=(3, 23, 1),
        padding='same'
    ))
    
    model.add(MaxPooling2D(pool_size=pool, padding='same'))

    model.add(Conv2D(
        filters=filters,
        kernel_size=3,
        activation='relu',
        padding='same'
    ))
    
    model.add(MaxPooling2D(pool_size=pool, padding='same'))
  
    model.add(Dense(
        hp.Int('dense_units', min_value=100, max_value=150, step=5),
        activation='relu'))
    

    x,y = model.output_shape[2], model.output_shape[3]

    model.add(Reshape((x, y)))

    model.add(LSTM(
        units=hp.Int('lstm_units', min_value=10, max_value=50, step=10),
        activation='relu',
    ))
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(units=1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

# Function to fit the model and evaluate metrics
def fit_and_evaluate(data, scaler, window_size, close_index, project_name):
    data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])
    training_size = int(len(data) * 0.9)
    training_data = data[:training_size]
    test_data = data[training_size:]
    train_data = training_data[:int(len(training_data) * 0.9)]
    val_data = training_data[int(len(training_data) * 0.9):]

    X_train, y_train = create_dataset(train_data, window_size, close_index)
    X_test, y_test = create_dataset(test_data, window_size, close_index)
    X_val, y_val = create_dataset(val_data, window_size, close_index)
    X_train_full, y_train_full = create_dataset(training_data, window_size, close_index)

    X_train = X_train.reshape((X_train.shape[0], window_size, X_train.shape[2], 1))
    X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))
    X_val = X_val.reshape((X_val.shape[0], window_size, X_val.shape[2], 1))
    X_train_full = X_train_full.reshape((X_train_full.shape[0], window_size, X_train_full.shape[2], 1))

    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=200,
        executions_per_trial=1,
        directory='my_dir',
        project_name=project_name,
        overwrite=False
    )

    early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

    tuner.search(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), verbose=2, callbacks=[early_stopping])

    best_hps = tuner.get_best_hyperparameters()[0]

    accuracy_list, f1_list, precision_list, recall_list = [], [], [], []
    for _ in range(10):
        best_model = tuner.hypermodel.build(best_hps) 
        history = best_model.fit(X_train_full, y_train_full, epochs=250, batch_size=32, verbose=2)
        predictions = best_model.predict(X_test)
        test_pred = (predictions > 0.5).astype(int)

        accuracy_list.append(accuracy_score(y_test, test_pred))
        f1_list.append(f1_score(y_test, test_pred))
        precision_list.append(precision_score(y_test, test_pred))
        recall_list.append(recall_score(y_test, test_pred))

    return {
        'accuracy': np.mean(accuracy_list),
        'f1_score': np.mean(f1_list),
        'precision': np.mean(precision_list),
        'recall': np.mean(recall_list)
    }

# Process each period and store results
for period, file_path in file_paths.items():
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    project_name = f'CNNLSTM-3D-CLASS-{period}'
    results[period] = fit_and_evaluate(data, scaler, window_size, close_index, project_name)

# Create a DataFrame from the results
results_df = pd.DataFrame(results).T
print(results_df)

# Save the results to a CSV file
results_df.to_csv('model_performance_summary_cnn_lstm_classification.csv')


Trial 200 Complete [00h 00m 42s]
val_accuracy: 0.5106382966041565

Best val_accuracy So Far: 0.6170212626457214
Total elapsed time: 02h 24m 56s
Epoch 1/250
31/31 - 7s - 240ms/step - accuracy: 0.4964 - loss: 0.6933
Epoch 2/250
31/31 - 0s - 4ms/step - accuracy: 0.5275 - loss: 0.6915
Epoch 3/250
31/31 - 0s - 3ms/step - accuracy: 0.5286 - loss: 0.6921
Epoch 4/250
31/31 - 0s - 4ms/step - accuracy: 0.5358 - loss: 0.6922
Epoch 5/250
31/31 - 0s - 4ms/step - accuracy: 0.5317 - loss: 0.6921
Epoch 6/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6914
Epoch 7/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6898
Epoch 8/250
31/31 - 0s - 4ms/step - accuracy: 0.5337 - loss: 0.6907
Epoch 9/250
31/31 - 0s - 4ms/step - accuracy: 0.5265 - loss: 0.6905
Epoch 10/250
31/31 - 0s - 4ms/step - accuracy: 0.5337 - loss: 0.6883
Epoch 11/250
31/31 - 0s - 5ms/step - accuracy: 0.5452 - loss: 0.6898
Epoch 12/250
31/31 - 0s - 4ms/step - accuracy: 0.5628 - loss: 0.6834
Epoch 13/250
31/31 - 0s - 4ms/step 

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 - 7s - 215ms/step - accuracy: 0.5140 - loss: 0.6928
Epoch 2/250
31/31 - 0s - 5ms/step - accuracy: 0.5327 - loss: 0.6921
Epoch 3/250
31/31 - 0s - 4ms/step - accuracy: 0.5348 - loss: 0.6920
Epoch 4/250
31/31 - 0s - 4ms/step - accuracy: 0.5317 - loss: 0.6911
Epoch 5/250
31/31 - 0s - 6ms/step - accuracy: 0.5254 - loss: 0.6939
Epoch 6/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6905
Epoch 7/250
31/31 - 0s - 4ms/step - accuracy: 0.5317 - loss: 0.6918
Epoch 8/250
31/31 - 0s - 4ms/step - accuracy: 0.5337 - loss: 0.6905
Epoch 9/250
31/31 - 0s - 5ms/step - accuracy: 0.5296 - loss: 0.6891
Epoch 10/250
31/31 - 0s - 4ms/step - accuracy: 0.5275 - loss: 0.6902
Epoch 11/250
31/31 - 0s - 4ms/step - accuracy: 0.5265 - loss: 0.6873
Epoch 12/250
31/31 - 0s - 4ms/step - accuracy: 0.5337 - loss: 0.6866
Epoch 13/250
31/31 - 0s - 4ms/step - accuracy: 0.5545 - loss: 0.6886
Epoch 14/250
31/31 - 0s - 4ms/step - accuracy: 0.5410 - loss: 0.6859
Epoch 15/250
31/31 - 0s - 4ms/step - accuracy: 0.5670

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 - 5s - 171ms/step - accuracy: 0.5192 - loss: 0.6944
Epoch 2/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6927
Epoch 3/250
31/31 - 0s - 4ms/step - accuracy: 0.5358 - loss: 0.6904
Epoch 4/250
31/31 - 0s - 4ms/step - accuracy: 0.5119 - loss: 0.6943
Epoch 5/250
31/31 - 0s - 4ms/step - accuracy: 0.5337 - loss: 0.6920
Epoch 6/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6910
Epoch 7/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6910
Epoch 8/250
31/31 - 0s - 5ms/step - accuracy: 0.5306 - loss: 0.6911
Epoch 9/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6902
Epoch 10/250
31/31 - 0s - 4ms/step - accuracy: 0.5317 - loss: 0.6880
Epoch 11/250
31/31 - 0s - 4ms/step - accuracy: 0.5182 - loss: 0.6917
Epoch 12/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6894
Epoch 13/250
31/31 - 0s - 4ms/step - accuracy: 0.5275 - loss: 0.6897
Epoch 14/250
31/31 - 0s - 4ms/step - accuracy: 0.5317 - loss: 0.6865
Epoch 15/250
31/31 - 0s - 7ms/step - accuracy: 0.5400

...

31/31 - 4s - 141ms/step - accuracy: 0.5036 - loss: 0.6933
Epoch 2/250
31/31 - 0s - 5ms/step - accuracy: 0.5171 - loss: 0.6926
Epoch 3/250
31/31 - 0s - 5ms/step - accuracy: 0.5254 - loss: 0.6923
Epoch 4/250
31/31 - 0s - 6ms/step - accuracy: 0.5306 - loss: 0.6920
Epoch 5/250
31/31 - 0s - 6ms/step - accuracy: 0.5306 - loss: 0.6910
Epoch 6/250
31/31 - 0s - 5ms/step - accuracy: 0.5306 - loss: 0.6897
Epoch 7/250
31/31 - 0s - 5ms/step - accuracy: 0.5306 - loss: 0.6909
Epoch 8/250
31/31 - 0s - 6ms/step - accuracy: 0.5265 - loss: 0.6902
Epoch 9/250
31/31 - 0s - 5ms/step - accuracy: 0.5254 - loss: 0.6878
Epoch 10/250
31/31 - 0s - 5ms/step - accuracy: 0.5431 - loss: 0.6896
Epoch 11/250
31/31 - 0s - 5ms/step - accuracy: 0.5400 - loss: 0.6877
Epoch 12/250
31/31 - 0s - 5ms/step - accuracy: 0.5389 - loss: 0.6876
Epoch 13/250
31/31 - 0s - 6ms/step - accuracy: 0.5587 - loss: 0.6881
Epoch 14/250
31/31 - 0s - 6ms/step - accuracy: 0.5670 - loss: 0.6843
Epoch 15/250
31/31 - 0s - 5ms/step - accuracy: 0.5431

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 - 4s - 144ms/step - accuracy: 0.5016 - loss: 0.6933
Epoch 2/250
31/31 - 0s - 8ms/step - accuracy: 0.5296 - loss: 0.6919
Epoch 3/250
31/31 - 0s - 5ms/step - accuracy: 0.5306 - loss: 0.6923
Epoch 4/250
31/31 - 0s - 5ms/step - accuracy: 0.5317 - loss: 0.6914
Epoch 5/250
31/31 - 0s - 5ms/step - accuracy: 0.5306 - loss: 0.6917
Epoch 6/250
31/31 - 0s - 5ms/step - accuracy: 0.5306 - loss: 0.6921
Epoch 7/250
31/31 - 0s - 5ms/step - accuracy: 0.5306 - loss: 0.6898
Epoch 8/250
31/31 - 0s - 5ms/step - accuracy: 0.5286 - loss: 0.6913
Epoch 9/250
31/31 - 0s - 5ms/step - accuracy: 0.5410 - loss: 0.6893
Epoch 10/250
31/31 - 0s - 5ms/step - accuracy: 0.5369 - loss: 0.6869
Epoch 11/250
31/31 - 0s - 6ms/step - accuracy: 0.5431 - loss: 0.6894
Epoch 12/250
31/31 - 0s - 5ms/step - accuracy: 0.5535 - loss: 0.6858
Epoch 13/250
31/31 - 0s - 5ms/step - accuracy: 0.5566 - loss: 0.6886
Epoch 14/250
31/31 - 0s - 5ms/step - accuracy: 0.5618 - loss: 0.6804
Epoch 15/250
31/31 - 0s - 5ms/step - accuracy: 0.5514

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 - 4s - 131ms/step - accuracy: 0.5234 - loss: 0.6929
Epoch 2/250
31/31 - 0s - 4ms/step - accuracy: 0.5223 - loss: 0.6926
Epoch 3/250
31/31 - 0s - 4ms/step - accuracy: 0.5296 - loss: 0.6935
Epoch 4/250
31/31 - 0s - 4ms/step - accuracy: 0.5286 - loss: 0.6921
Epoch 5/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6914
Epoch 6/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6915
Epoch 7/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6924
Epoch 8/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6901
Epoch 9/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6908
Epoch 10/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6904
Epoch 11/250
31/31 - 0s - 4ms/step - accuracy: 0.5327 - loss: 0.6905
Epoch 12/250
31/31 - 0s - 4ms/step - accuracy: 0.5306 - loss: 0.6916
Epoch 13/250
31/31 - 0s - 4ms/step - accuracy: 0.5337 - loss: 0.6854
Epoch 14/250
31/31 - 0s - 4ms/step - accuracy: 0.5410 - loss: 0.6865
Epoch 15/250
31/31 - 0s - 4ms/step - accuracy: 0.5535

In [3]:
# Get the best hyperparameters for bear market 1
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNNLSTM-3D-CLASS-bear_market_1',
    overwrite=False
)

tuner.get_best_hyperparameters()[0].values
# save model for bear market 1
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('CNNLSTM-CLASS-bear_market_1.keras')

# Get the best hyperparameters for bear market 2
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNNLSTM-3D-CLASS-bear_market_2',
    overwrite=False
)

tuner.get_best_hyperparameters()[0].values
# save model for bear market 2
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('CNNLSTM-CLASS-bear_market_2.keras')

# Get the best hyperparameters for bull market 1
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNNLSTM-3D-CLASS-bull_market_1',
    overwrite=False
)

tuner.get_best_hyperparameters()[0].values
# save model for bull market 1
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('CNNLSTM-CLASS-bull_market_1.keras')

# Get the best hyperparameters for bull market 2
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNNLSTM-3D-CLASS-bull_market_2',
    overwrite=False
)

tuner.get_best_hyperparameters()[0].values
# save model for bull market 2
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('CNNLSTM-CLASS-bull_market_2.keras')


Reloading Tuner from my_dir/CNNLSTM-3D-CLASS-bear_market_1/tuner0.json


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Reloading Tuner from my_dir/CNNLSTM-3D-CLASS-bear_market_2/tuner0.json
Reloading Tuner from my_dir/CNNLSTM-3D-CLASS-bull_market_1/tuner0.json
Reloading Tuner from my_dir/CNNLSTM-3D-CLASS-bull_market_2/tuner0.json
